# Amazon SageMaker - Bring Your Own Container¶
### 学習用インスタンスでの学習
ここでは TensorFlow を使ったサンプルコードを題材に、Amazon SageMaker で独自コンテナを用いた機械学習モデルの学習方法を順を追って説明します。トレーニングスクリプトは既に SageMaker 向けに書き換えられた形で準備され、その上で、独自の学習用コンテナを活用します。 トレーニングスクリプトを SageMaker 向けに書き換える際は [Bring Your Own Model ハンズオンワークショップ](https://github.com/aws-samples/amazon-sagemaker-examples-jp/tree/master/workshop/lab_bring-your-own-model) をご参考に下さい。

また本ハンズオンで活用するデータは、[Bring Your Own Container のためのデータ準備](https://github.com/tkazusa/data_prep.ipynb)をお使い下さい。

In [1]:
import keras

import boto3
import sagemaker
from sagemaker import get_execution_role
from sagemaker.estimator import Estimator

sess = sagemaker.Session()

Using TensorFlow backend.


In [2]:
!docker build -t sm-tf-nightly-gpu container/.

Sending build context to Docker daemon  13.31kB
Step 1/4 : FROM tensorflow/tensorflow:nightly-gpu
 ---> ad7cd4026486
Step 2/4 : RUN pip install sagemaker-containers
 ---> Using cache
 ---> 28545073ce2f
Step 3/4 : COPY tf_codes /opt/ml/code/
 ---> f756d6396bd2
Step 4/4 : ENV SAGEMAKER_PROGRAM train.py
 ---> Running in 38a612216f83
Removing intermediate container 38a612216f83
 ---> b94cf342c274
Successfully built b94cf342c274
Successfully tagged sm-tf-nightly-gpu:latest


## 学習用インスタンスを用いた学習
### ECR への Docker イメージの登録
build した Docker イメージを Docker コンテナレジストリである Amazon ECR へ登録することで、SageMaker の学習時に活用できるようになります。

In [3]:
import boto3

# boto3の機能を使ってリポジトリ名に必要な情報を取得する
account_id = boto3.client('sts').get_caller_identity().get('Account')
region = boto3.session.Session().region_name
tag = ':latest'

ecr_repository = 'sm-tf-nightly-gpu'
image_uri = '{}.dkr.ecr.{}.amazonaws.com/{}'.format(account_id, region, ecr_repository + tag)

!$(aws ecr get-login --region $region --registry-ids $account_id --no-include-email)
 
# リポジトリの作成
# すでにある場合はこのコマンドは必要ない
!aws ecr create-repository --repository-name $ecr_repository
 
!docker build -t word2vec .
!docker tag {ecr_repository + tag} $image_uri
!docker push $image_uri

print('コンテナは {} へ登録されています。'.format(image_uri))

WARNING! Using --password via the CLI is insecure. Use --password-stdin.
WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store

Login Succeeded

An error occurred (RepositoryAlreadyExistsException) when calling the CreateRepository operation: The repository with name 'sm-tf-nightly-gpu' already exists in the registry with id '815969174475'
unable to prepare context: unable to evaluate symlinks in Dockerfile path: lstat /home/ec2-user/SageMaker/sagemaker-tensorflow-byoc/Dockerfile: no such file or directory
The push refers to repository [815969174475.dkr.ecr.us-east-1.amazonaws.com/sm-tf-nightly-gpu]

4cc0702b: Preparing 
dd9cc193: Preparing 
ff51c6a3: Preparing 
753ea774: Preparing 
b74783e3: Preparing 
8916be48: Preparing 
0a0d84e3: Preparing 
97f02d3e: Preparing 
45052e8d: Preparing 
eb49a301: Preparing 
f76fc560: P

### GPU を使った学習インスタンスでの学習
登録された Docker イメージと、S3 へアップロードされたデータを用いて学習を行います。

In [19]:
from sagemaker.tuner import CategoricalParameter, HyperparameterTuner

role = get_execution_role()
hyperparameters = {'batch_size': 64,'epochs': 30}

bucket = '20200222-tensorflow-byoc'
output_path = 's3://{}/output'.format(bucket)

data_path = {'train': 's3://{}/data'.format(bucket)}

hyperparameter_ranges = {'optimizer': CategoricalParameter(['adam', 'sgd'])}


objective_metric_name = 'loss'
objective_type = 'Minimize'

metric_definitions = [{'Name': 'loss',
                       'Regex': 'loss: ([0-9\\.]+)'},
                      {'Name': 'accuracy',
                       'Regex': 'accuracy: ([0-9\\.]+)'}]

estimator = Estimator(image_name=image_uri,
                      role=role,
                      hyperparameters=hyperparameters,
                      train_instance_count=1,
                      train_instance_type='ml.p2.xlarge',
                      output_path=output_path)

tuner = HyperparameterTuner(estimator,
                            objective_metric_name,
                            hyperparameter_ranges,
                            metric_definitions,
                            max_jobs=2,
                            max_parallel_jobs=1,
                            objective_type=objective_type,
                            early_stopping_type='Auto')


tuner.fit(data_path)

## ハイパラチューニングジョブの可視化

In [23]:
hpo_job_name = 'sm-tf-nightly-gpu-200224-0544'
tuning_job = sagemaker.HyperparameterTuningJobAnalytics(hpo_job_name)
df_hpo = tuning_job.dataframe()

# ジョブが始まってたら値取れる？
df_hpo

,FinalObjectiveValue,TrainingElapsedTimeSeconds,TrainingEndTime,TrainingJobName,TrainingJobStatus,TrainingStartTime,optimizer
0,0.0168,215.0,2020-02-24 05:56:51+00:00,sm-tf-nightly-gpu-200224-0544-002-b1aebc9a,Completed,2020-02-24 05:53:16+00:00,adam
1,1.1523,212.0,2020-02-24 05:50:38+00:00,sm-tf-nightly-gpu-200224-0544-001-ad72226e,Completed,2020-02-24 05:47:06+00:00,sgd


## See TuningJob results vs time

In [25]:
import bokeh
import bokeh.io
bokeh.io.output_notebook()
from bokeh.plotting import figure, show
from bokeh.models import HoverTool


class HoverHelper():

    def __init__(self, tuning_analytics):
        self.tuner = tuning_analytics

    def hovertool(self):
        tooltips = [
            ("FinalObjectiveValue", "@FinalObjectiveValue"),
            ("TrainingJobName", "@TrainingJobName"),
        ]
        for k in self.tuner.tuning_ranges.keys():
            tooltips.append( (k, "@{%s}" % k) )

        ht = HoverTool(tooltips=tooltips)
        return ht

    def tools(self, standard_tools='pan,crosshair,wheel_zoom,zoom_in,zoom_out,undo,reset'):
        return [self.hovertool(), standard_tools]

hover = HoverHelper(tuning_job)

p = figure(plot_width=900, plot_height=400, tools=hover.tools(), x_axis_type='datetime')
p.circle(source=df_hpo, x='TrainingStartTime', y='FinalObjectiveValue')
show(p)

Loading BokehJS ...

In [ ]:
tuner